INSERT OVERWRITE LOCAL DIRECTORY './demo' 
      ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
      SELECT * FROM Cataloguee;
      

In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import split, substring, length, avg, upper,coalesce



In [46]:
spark = SparkSession.builder \
    .appName("mapreduce") \
    .getOrCreate()

In [47]:
df_catalogue = spark.read \
    .format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load("file:///home/vagrant/export/000000_0")

In [48]:
df_catalogue.show(2)


+-----+------+---+-----------+---+---+-----+-----+-----+
|  _c0|   _c1|_c2|        _c3|_c4|_c5|  _c6|  _c7|  _c8|
+-----+------+---+-----------+---+---+-----+-----+-----+
|Volvo|S80 T6|272|tr�s longue|  5|  5|blanc|false|50500|
|Volvo|S80 T6|272|tr�s longue|  5|  5| noir|false|50500|
+-----+------+---+-----------+---+---+-----+-----+-----+
only showing top 2 rows



In [49]:
df_catalogue = df_catalogue.withColumnRenamed("_c0", "Marque") \
       .withColumnRenamed("_c1", "Modele") \
       .withColumnRenamed("_c2", "Puissance") \
       .withColumnRenamed("_c3", "Longueur") \
       .withColumnRenamed("_c4", "nbPlaces") \
       .withColumnRenamed("_c5", "nbPortes") \
       .withColumnRenamed("_c6", "Couleur") \
       .withColumnRenamed("_c7", "Occasion") \
       .withColumnRenamed("_c8", "Prix")

In [6]:
df_catalogue.show(2)

+------+------+---------+-----------+--------+--------+-------+--------+-----+
|Marque|Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion| Prix|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
| Volvo|S80 T6|      272|tr�s longue|       5|       5|  blanc|   false|50500|
| Volvo|S80 T6|      272|tr�s longue|       5|       5|   noir|   false|50500|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
only showing top 2 rows



In [50]:
df_catalogue = df_catalogue.withColumn("Longueur", regexp_replace(col("Longueur"), "tr�s", "tres"))




In [51]:
df_catalogue.show(2)

+------+------+---------+-----------+--------+--------+-------+--------+-----+
|Marque|Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion| Prix|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
| Volvo|S80 T6|      272|tres longue|       5|       5|  blanc|   false|50500|
| Volvo|S80 T6|      272|tres longue|       5|       5|   noir|   false|50500|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
only showing top 2 rows



In [55]:
df_co2 = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///home/vagrant/CO2.csv")

In [10]:
df_co2.show(2)

+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI E-TRON SPORT...|        -6000|              0|        319|
|  3|AUDI E-TRON SPORT...|        -6000|              0|        356|
+---+--------------------+-------------+---------------+-----------+
only showing top 2 rows



In [56]:
df_co2 = df_co2.drop("_c0")

In [12]:
df_co2.show(2)

+--------------------+-------------+---------------+-----------+
|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+--------------------+-------------+---------------+-----------+
|AUDI E-TRON SPORT...|        -6000|              0|        319|
|AUDI E-TRON SPORT...|        -6000|              0|        356|
+--------------------+-------------+---------------+-----------+
only showing top 2 rows



In [57]:
df_co2 = df_co2.withColumnRenamed("Marque / Modele", "marque_modele") \
               .withColumnRenamed("Bonus / Malus", "bonus_malus") \
               .withColumnRenamed("Rejets CO2 g/km", "rejets_co2_g_km") \
               .withColumnRenamed("Cout enerie", "cout_energie")

In [14]:
df_co2.show(2)

+--------------------+-----------+---------------+------------+
|       marque_modele|bonus_malus|rejets_co2_g_km|cout_energie|
+--------------------+-----------+---------------+------------+
|AUDI E-TRON SPORT...|      -6000|              0|         319|
|AUDI E-TRON SPORT...|      -6000|              0|         356|
+--------------------+-----------+---------------+------------+
only showing top 2 rows



In [58]:
df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), "Ã©", "e"))
df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), "\"", ""))

In [60]:
df_co2.show()

+--------------------+-----------+---------------+------------+
|       marque_modele|bonus_malus|rejets_co2_g_km|cout_energie|
+--------------------+-----------+---------------+------------+
|AUDI E-TRON SPORT...|      -6000|              0|         319|
|AUDI E-TRON SPORT...|      -6000|              0|         356|
|AUDI E-TRON 55 (4...|      -6000|              0|         357|
|AUDI E-TRON 50 (3...|      -6000|              0|         356|
|       BMW i3 120 Ah|      -6000|              0|         204|
|      BMW i3s 120 Ah|      -6000|              0|         204|
|    CITROEN BERLINGO|      -6000|              0|         203|
|      CITROEN C-ZERO|      -6000|              0|         491|
|DS DS3 CROSSBACK ...|      -6000|              0|         251|
|HYUNDAI KONA elec...|      -6000|              0|         205|
|HYUNDAI KONA elec...|      -6000|              0|         205|
|JAGUAR I-PACE EV4...|      -6000|              0|         271|
|KIA e-NIRO Moteur...|      -6000|      

In [59]:
df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), "Ã©", "e"))
df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), "Ã¨", "e"))
df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), "Ã\xa0", "a"))


In [62]:
df_co2 = df_co2.withColumn('marque', split(df_co2['marque_modele'], ' ')[0])


In [19]:
df_co2.show(2)

+--------------------+-----------+---------------+------------+------+
|       marque_modele|bonus_malus|rejets_co2_g_km|cout_energie|marque|
+--------------------+-----------+---------------+------------+------+
|AUDI E-TRON SPORT...|      -6000|              0|         319|  AUDI|
|AUDI E-TRON SPORT...|      -6000|              0|         356|  AUDI|
+--------------------+-----------+---------------+------------+------+
only showing top 2 rows



In [63]:
l=df_co2.select("marque").rdd.flatMap(lambda x: x).collect()
for i in l:
    df_co2 = df_co2.withColumn("marque_modele", regexp_replace(col("marque_modele"), i, ""))


df_co2.show(1000)


In [207]:
df_co2.show(2)


+--------------------+-----------+---------------+------------+------+
|       marque_modele|bonus_malus|rejets_co2_g_km|cout_energie|marque|
+--------------------+-----------+---------------+------------+------+
| E-TRON SPORTBACK...|      -6000|              0|         319|  AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|  AUDI|
+--------------------+-----------+---------------+------------+------+
only showing top 2 rows



In [64]:
df_co2 = df_co2.withColumnRenamed("marque_modele", "modele")

In [65]:
df_co2.show()

+--------------------+-----------+---------------+------------+--------+
|              modele|bonus_malus|rejets_co2_g_km|cout_energie|  marque|
+--------------------+-----------+---------------+------------+--------+
| E-TRON SPORTBACK...|      -6000|              0|         319|    AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|    AUDI|
| E-TRON 55 (408ch...|      -6000|              0|         357|    AUDI|
| E-TRON 50 (313ch...|      -6000|              0|         356|    AUDI|
|           i3 120 Ah|      -6000|              0|         204|     BMW|
|          i3s 120 Ah|      -6000|              0|         204|     BMW|
|            BERLINGO|      -6000|              0|         203| CITROEN|
|              C-ZERO|      -6000|              0|         491| CITROEN|
| 3 CROSSBACK E-Te...|      -6000|              0|         251|      DS|
| KONA electric 64...|      -6000|              0|         205| HYUNDAI|
| KONA electric 39...|      -6000|              0| 

In [210]:
df_co2.show(1000)

+--------------------+-----------+---------------+------------+----------+
|              modele|bonus_malus|rejets_co2_g_km|cout_energie|    marque|
+--------------------+-----------+---------------+------------+----------+
| E-TRON SPORTBACK...|      -6000|              0|         319|      AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|      AUDI|
| E-TRON 55 (408ch...|      -6000|              0|         357|      AUDI|
| E-TRON 50 (313ch...|      -6000|              0|         356|      AUDI|
|           i3 120 Ah|      -6000|              0|         204|       BMW|
|          i3s 120 Ah|      -6000|              0|         204|       BMW|
|            BERLINGO|      -6000|              0|         203|   CITROEN|
|              C-ZERO|      -6000|              0|         491|   CITROEN|
| 3 CROSSBACK E-Te...|      -6000|              0|         251|        DS|
| KONA electric 64...|      -6000|              0|         205|   HYUNDAI|
| KONA electric 39...|   

In [66]:
df_co2 = df_co2.withColumn("bonus_malus", col("bonus_malus").cast("integer"))
df_co2 = df_co2.withColumn("rejets_co2_g_km", col("rejets_co2_g_km").cast("integer"))
df_co2 = df_co2.withColumn("cout_energie", col("cout_energie").cast("integer"))

In [67]:
df_co2.show(1000)

+--------------------+-----------+---------------+------------+----------+
|              modele|bonus_malus|rejets_co2_g_km|cout_energie|    marque|
+--------------------+-----------+---------------+------------+----------+
| E-TRON SPORTBACK...|      -6000|              0|         319|      AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|      AUDI|
| E-TRON 55 (408ch...|      -6000|              0|         357|      AUDI|
| E-TRON 50 (313ch...|      -6000|              0|         356|      AUDI|
|           i3 120 Ah|      -6000|              0|         204|       BMW|
|          i3s 120 Ah|      -6000|              0|         204|       BMW|
|            BERLINGO|      -6000|              0|         203|   CITROEN|
|              C-ZERO|      -6000|              0|         491|   CITROEN|
| 3 CROSSBACK E-Te...|      -6000|              0|         251|        DS|
| KONA electric 64...|      -6000|              0|         205|   HYUNDAI|
| KONA electric 39...|   

In [68]:
df_co2_agg = df_co2.groupBy("Marque").agg(avg("Bonus_Malus").alias("avg_Bonus_Malus"),
                                            avg("Rejets_CO2_g_km").alias("avg_Rejets_CO2_g_km"),
                                            avg("Cout_Energie").alias("avg_Cout_Energie"))

In [70]:
df_co2_agg.show(1000)

+----------+-------------------+-------------------+------------------+
|    Marque|    avg_Bonus_Malus|avg_Rejets_CO2_g_km|  avg_Cout_Energie|
+----------+-------------------+-------------------+------------------+
|  MERCEDES| 7790.5864406779665|  187.6271186440678| 749.9796610169492|
|   PORSCHE|                0.0|  69.85714285714286| 89.71428571428571|
|   HYUNDAI|            -4000.0|  8.666666666666666|             151.0|
|    TOYOTA|                0.0|               32.0|              43.0|
|     SKODA| -666.6666666666666| 27.555555555555557| 98.88888888888889|
|    NISSAN|             5802.4|              160.0|             681.2|
|      LAND|                0.0|               69.0|              78.0|
|   CITROEN|            -6000.0|                0.0|             347.0|
|   BENTLEY|                0.0|               84.0|             102.0|
|      AUDI|            -2400.0|               26.1|             191.6|
|      MINI|            -3000.0|               21.5|            

In [215]:
df_catalogue.show(1000)

+----------+----------------+---------+-----------+--------+--------+-------+--------+------+
|    Marque|          Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion|  Prix|
+----------+----------------+---------+-----------+--------+--------+-------+--------+------+
|     Volvo|          S80 T6|      272|tres longue|       5|       5|  blanc|   false| 50500|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|   noir|   false| 50500|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|  rouge|   false| 50500|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|   gris|    true| 35350|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|   bleu|    true| 35350|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|   gris|   false| 50500|
|     Volvo|          S80 T6|      272|tres longue|       5|       5|   bleu|   false| 50500|
|     Volvo|          S80 T6|      272|tres longue|       5|

In [69]:
    df_catalogue = df_catalogue.withColumn("Marque", regexp_replace(col("Marque"),"Hyunda�", "Hyundai"))


In [217]:
df_co2.show(1000)

+--------------------+-----------+---------------+------------+----------+
|              modele|bonus_malus|rejets_co2_g_km|cout_energie|    marque|
+--------------------+-----------+---------------+------------+----------+
| E-TRON SPORTBACK...|      -6000|              0|         319|      AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|      AUDI|
| E-TRON 55 (408ch...|      -6000|              0|         357|      AUDI|
| E-TRON 50 (313ch...|      -6000|              0|         356|      AUDI|
|           i3 120 Ah|      -6000|              0|         204|       BMW|
|          i3s 120 Ah|      -6000|              0|         204|       BMW|
|            BERLINGO|      -6000|              0|         203|   CITROEN|
|              C-ZERO|      -6000|              0|         491|   CITROEN|
| 3 CROSSBACK E-Te...|      -6000|              0|         251|        DS|
| KONA electric 64...|      -6000|              0|         205|   HYUNDAI|
| KONA electric 39...|   

In [71]:
df_catalogue = df_catalogue.withColumn("Marque", upper(df_catalogue["Marque"]))
df_catalogue = df_catalogue.withColumn("Modele", upper(df_catalogue["Modele"]))
df_co2 = df_co2.withColumn("marque", upper(df_co2["marque"]))
df_co2 = df_co2.withColumn("modele", upper(df_co2["modele"]))




In [221]:
df_catalogue.show(5)

+------+------+---------+-----------+--------+--------+-------+--------+-----+
|Marque|Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion| Prix|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
| VOLVO|S80 T6|      272|tres longue|       5|       5|  blanc|   false|50500|
| VOLVO|S80 T6|      272|tres longue|       5|       5|   noir|   false|50500|
| VOLVO|S80 T6|      272|tres longue|       5|       5|  rouge|   false|50500|
| VOLVO|S80 T6|      272|tres longue|       5|       5|   gris|    true|35350|
| VOLVO|S80 T6|      272|tres longue|       5|       5|   bleu|    true|35350|
+------+------+---------+-----------+--------+--------+-------+--------+-----+
only showing top 5 rows



In [222]:
df_co2.show(5)

+--------------------+-----------+---------------+------------+------+
|              modele|bonus_malus|rejets_co2_g_km|cout_energie|marque|
+--------------------+-----------+---------------+------------+------+
| E-TRON SPORTBACK...|      -6000|              0|         319|  AUDI|
| E-TRON SPORTBACK...|      -6000|              0|         356|  AUDI|
| E-TRON 55 (408CH...|      -6000|              0|         357|  AUDI|
| E-TRON 50 (313CH...|      -6000|              0|         356|  AUDI|
|           I3 120 AH|      -6000|              0|         204|   BMW|
+--------------------+-----------+---------------+------------+------+
only showing top 5 rows



In [72]:
df_join = df_catalogue.join(df_co2, ["Marque", "Modele"], "left")


In [28]:
df_join.show(1000)


+----------+----------------+---------+-----------+--------+--------+-------+--------+------+-----------+---------------+------------+
|    Marque|          Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion|  Prix|bonus_malus|rejets_co2_g_km|cout_energie|
+----------+----------------+---------+-----------+--------+--------+-------+--------+------+-----------+---------------+------------+
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|  blanc|   false| 50500|       null|           null|        null|
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|   noir|   false| 50500|       null|           null|        null|
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|  rouge|   false| 50500|       null|           null|        null|
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|   gris|    true| 35350|       null|           null|        null|
|     VOLVO|          S80 T6|      272|tres longue|    

In [73]:
df_last = df_join.join(df_co2_agg, on="Marque", how="left")


In [226]:
df_last.show(1000)

+----------+----------------+---------+-----------+--------+--------+-------+--------+------+-----------+---------------+------------+-------------------+-------------------+------------------+
|    Marque|          Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion|  Prix|bonus_malus|rejets_co2_g_km|cout_energie|    avg_Bonus_Malus|avg_Rejets_CO2_g_km|  avg_Cout_Energie|
+----------+----------------+---------+-----------+--------+--------+-------+--------+------+-----------+---------------+------------+-------------------+-------------------+------------------+
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|  blanc|   false| 50500|       null|           null|        null|                0.0|  42.45454545454545| 72.72727272727273|
|     VOLVO|          S80 T6|      272|tres longue|       5|       5|   noir|   false| 50500|       null|           null|        null|                0.0|  42.45454545454545| 72.72727272727273|
|     VOLVO|          S80 T6| 

In [74]:
mydf = df_last.withColumn("bonus_malus", coalesce(df_last["bonus_malus"], df_last["avg_Bonus_Malus"]))
mydf = mydf.withColumn("rejets_co2_g_km", coalesce(mydf["rejets_co2_g_km"], mydf["avg_Rejets_CO2_g_km"]))
mydf = mydf.withColumn("cout_energie", coalesce(mydf["cout_energie"], mydf["avg_Cout_Energie"]))


In [75]:
mydf.show(6)

+------+------+---------+-----------+--------+--------+-------+--------+-----+-----------+-----------------+-----------------+---------------+-------------------+-----------------+
|Marque|Modele|Puissance|   Longueur|nbPlaces|nbPortes|Couleur|Occasion| Prix|bonus_malus|  rejets_co2_g_km|     cout_energie|avg_Bonus_Malus|avg_Rejets_CO2_g_km| avg_Cout_Energie|
+------+------+---------+-----------+--------+--------+-------+--------+-----+-----------+-----------------+-----------------+---------------+-------------------+-----------------+
| VOLVO|S80 T6|      272|tres longue|       5|       5|  blanc|   false|50500|        0.0|42.45454545454545|72.72727272727273|            0.0|  42.45454545454545|72.72727272727273|
| VOLVO|S80 T6|      272|tres longue|       5|       5|   noir|   false|50500|        0.0|42.45454545454545|72.72727272727273|            0.0|  42.45454545454545|72.72727272727273|
| VOLVO|S80 T6|      272|tres longue|       5|       5|  rouge|   false|50500|        0.0|42.45

In [76]:
mydf.columns

['Marque',
 'Modele',
 'Puissance',
 'Longueur',
 'nbPlaces',
 'nbPortes',
 'Couleur',
 'Occasion',
 'Prix',
 'bonus_malus',
 'rejets_co2_g_km',
 'cout_energie',
 'avg_Bonus_Malus',
 'avg_Rejets_CO2_g_km',
 'avg_Cout_Energie']

In [77]:
mydf = mydf.drop("avg_Bonus_Malus", "avg_Rejets_CO2_g_km", "avg_Cout_Energie")


In [78]:
mydf.columns

['Marque',
 'Modele',
 'Puissance',
 'Longueur',
 'nbPlaces',
 'nbPortes',
 'Couleur',
 'Occasion',
 'Prix',
 'bonus_malus',
 'rejets_co2_g_km',
 'cout_energie']

In [79]:
mydf.write.csv("file:///home/vagrant/exportdemo/merged")

CREATE TABLE catalogue (
    Marque string,
    Modele string,
    Puissance int,
    Longueur string,
    nbPlaces int,
    nbPortes int,
    Couleur string,
    Occasion boolean,
    Prix int,
    bonus_malus float,
    rejets_co2_g_km float,
    cout_energie float
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

LOAD DATA LOCAL INPATH '/home/vagrant/export/merged/t.csv' INTO TABLE catalogue;